# Questions to the data
1. Does the number of hauses in all houses groups remain stable or there were a construction side finished in the given time frame?
2. What was the weather in the region?
3. What was the price for gas/oil/elictricity?

In [47]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import List, Tuple, Any

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklego.preprocessing import RepeatingBasisFunction
import warnings
warnings.filterwarnings("ignore")

In [48]:
data = pd.read_csv('../data/public_data/train.csv')
final_index = data['pseudo_id']

In [49]:
# drop index for feature preparation
data_ = data.drop(columns='pseudo_id')
# convert dates to pandas datetime
data_.columns = [datetime.strptime(c, "%Y-%m-%d %H:%M:%S") for c in data_.columns]
data_.head()

,2017-01-01 00:00:00,2017-01-01 00:30:00,2017-01-01 01:00:00,2017-01-01 01:30:00,2017-01-01 02:00:00,2017-01-01 02:30:00,2017-01-01 03:00:00,2017-01-01 03:30:00,2017-01-01 04:00:00,2017-01-01 04:30:00,...,2019-08-28 19:00:00,2019-08-28 19:30:00,2019-08-28 20:00:00,2019-08-28 20:30:00,2019-08-28 21:00:00,2019-08-28 21:30:00,2019-08-28 22:00:00,2019-08-28 22:30:00,2019-08-28 23:00:00,2019-08-28 23:30:00
0,45.023,39.9850,36.5695,34.7480,35.972,38.439,36.591,36.3155,32.6605,0.1420,...,24.2880,23.994,26.1995,25.0270,23.0665,26.093,23.4295,25.4715,26.246,22.6020
1,2.931,1.6410,2.2600,2.2730,2.651,3.137,2.532,3.1420,2.5280,0.0000,...,2.5700,1.446,1.5230,1.5630,2.5880,2.190,1.4860,2.5270,2.288,1.7940
2,11.014,12.6525,10.8240,13.7485,12.383,12.342,13.413,11.4840,11.5105,0.0455,...,6.3565,5.766,5.4955,5.0885,6.8140,7.492,5.7705,6.8240,6.072,6.7205
3,55.813,49.0400,49.0950,41.1330,45.660,48.477,50.539,45.7370,42.6800,0.0000,...,32.6460,30.439,30.2470,31.2660,34.3390,33.076,33.1080,33.7260,30.009,34.8400
4,26.925,28.1180,25.6000,28.0910,26.530,23.858,26.556,27.7140,23.1740,0.0000,...,13.3980,13.280,13.7340,13.6060,14.7000,16.290,15.1240,15.3650,14.360,13.9350


In [50]:
freq = "H"
data_ = data_.T.resample(freq).sum()
data_.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
2017-01-01 00:00:00,85.0080,4.572,23.6665,104.853,55.043,21.670,8.517,50.1745,74.8950,22.242,...,0.368,11.430,9.319,1.745,5.848,1.232,3.041,6.9575,2.0065,6.693
2017-01-01 01:00:00,71.3175,4.533,24.5725,90.228,53.691,20.802,9.665,47.1655,63.5045,21.320,...,0.680,11.414,9.950,1.807,8.823,0.669,2.868,5.6150,1.4100,5.698
2017-01-01 02:00:00,74.4110,5.788,24.7250,94.137,50.388,20.309,10.970,51.7245,62.5770,23.942,...,1.400,14.907,8.828,1.650,8.535,0.645,2.892,6.6505,1.5015,5.953
2017-01-01 03:00:00,72.9065,5.674,24.8970,96.276,54.270,19.099,9.228,49.3040,64.9425,21.885,...,0.803,12.920,8.203,1.362,5.721,1.418,3.190,7.4195,1.2500,6.856
2017-01-01 04:00:00,32.8025,2.528,11.5560,42.680,23.174,8.557,3.721,23.9510,26.9285,10.102,...,0.205,6.317,3.524,0.869,2.667,0.303,1.803,3.7660,0.7975,1.632


In [51]:
# features["hour"] = data.index.hour
data_.columns = [c+1 for c in range(len(data_.columns))]
data_.head()

,1,2,3,4,5,6,7,8,9,10,...,52,53,54,55,56,57,58,59,60,61
2017-01-01 00:00:00,85.0080,4.572,23.6665,104.853,55.043,21.670,8.517,50.1745,74.8950,22.242,...,0.368,11.430,9.319,1.745,5.848,1.232,3.041,6.9575,2.0065,6.693
2017-01-01 01:00:00,71.3175,4.533,24.5725,90.228,53.691,20.802,9.665,47.1655,63.5045,21.320,...,0.680,11.414,9.950,1.807,8.823,0.669,2.868,5.6150,1.4100,5.698
2017-01-01 02:00:00,74.4110,5.788,24.7250,94.137,50.388,20.309,10.970,51.7245,62.5770,23.942,...,1.400,14.907,8.828,1.650,8.535,0.645,2.892,6.6505,1.5015,5.953
2017-01-01 03:00:00,72.9065,5.674,24.8970,96.276,54.270,19.099,9.228,49.3040,64.9425,21.885,...,0.803,12.920,8.203,1.362,5.721,1.418,3.190,7.4195,1.2500,6.856
2017-01-01 04:00:00,32.8025,2.528,11.5560,42.680,23.174,8.557,3.721,23.9510,26.9285,10.102,...,0.205,6.317,3.524,0.869,2.667,0.303,1.803,3.7660,0.7975,1.632


In [52]:
# check wether datetime in ascending order , it is important for time series
print(data_.index.is_monotonic)

True


In [53]:
data_[(data_.index >='2017-02-08 00:00:00') & (data_.index <= '2017-02-14 23:00:00')]

,1,2,3,4,5,6,7,8,9,10,...,52,53,54,55,56,57,58,59,60,61
2017-02-08 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-02-08 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-02-08 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-02-08 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-02-08 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-02-14 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-02-14 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-02-14 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-02-14 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
# Set dates for development phase
new_date_range = pd.date_range(start="2019-04-01", end="2019-09-05", freq="H")
data_ = data_.reindex(new_date_range)

In [55]:
new_date_range

DatetimeIndex(['2019-04-01 00:00:00', '2019-04-01 01:00:00',
               '2019-04-01 02:00:00', '2019-04-01 03:00:00',
               '2019-04-01 04:00:00', '2019-04-01 05:00:00',
               '2019-04-01 06:00:00', '2019-04-01 07:00:00',
               '2019-04-01 08:00:00', '2019-04-01 09:00:00',
               ...
               '2019-09-04 15:00:00', '2019-09-04 16:00:00',
               '2019-09-04 17:00:00', '2019-09-04 18:00:00',
               '2019-09-04 19:00:00', '2019-09-04 20:00:00',
               '2019-09-04 21:00:00', '2019-09-04 22:00:00',
               '2019-09-04 23:00:00', '2019-09-05 00:00:00'],
              dtype='datetime64[ns]', length=3769, freq='H')

In [56]:
data_=data_.drop(data_.index[-1])
data_.tail()

,1,2,3,4,5,6,7,8,9,10,...,52,53,54,55,56,57,58,59,60,61
2019-09-04 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-04 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-04 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-04 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-04 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# saving indexes for next steps
idx_test_date = data_.index[(data_[1] == 0) | (data_[1].isna())]
idx_test_date

DatetimeIndex(['2019-04-29 00:00:00', '2019-04-29 01:00:00',
               '2019-04-29 02:00:00', '2019-04-29 03:00:00',
               '2019-04-29 04:00:00', '2019-04-29 05:00:00',
               '2019-04-29 06:00:00', '2019-04-29 07:00:00',
               '2019-04-29 08:00:00', '2019-04-29 09:00:00',
               ...
               '2019-09-04 14:00:00', '2019-09-04 15:00:00',
               '2019-09-04 16:00:00', '2019-09-04 17:00:00',
               '2019-09-04 18:00:00', '2019-09-04 19:00:00',
               '2019-09-04 20:00:00', '2019-09-04 21:00:00',
               '2019-09-04 22:00:00', '2019-09-04 23:00:00'],
              dtype='datetime64[ns]', length=672, freq=None)

In [58]:
idx_test_date[0:168]

DatetimeIndex(['2019-04-29 00:00:00', '2019-04-29 01:00:00',
               '2019-04-29 02:00:00', '2019-04-29 03:00:00',
               '2019-04-29 04:00:00', '2019-04-29 05:00:00',
               '2019-04-29 06:00:00', '2019-04-29 07:00:00',
               '2019-04-29 08:00:00', '2019-04-29 09:00:00',
               ...
               '2019-05-05 14:00:00', '2019-05-05 15:00:00',
               '2019-05-05 16:00:00', '2019-05-05 17:00:00',
               '2019-05-05 18:00:00', '2019-05-05 19:00:00',
               '2019-05-05 20:00:00', '2019-05-05 21:00:00',
               '2019-05-05 22:00:00', '2019-05-05 23:00:00'],
              dtype='datetime64[ns]', length=168, freq=None)

In [59]:
data_ = data_.fillna(method="ffill")

In [60]:
#plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':160})
#data_[1].plot()
#plt.title('Energy use forecasts for houshold group' + " 1")
#plt.show()

In [61]:
data_\
    .isna()\
    .sum()\
    .to_frame()\
    .assign(perc = lambda row: 100 * row[0] / data_.shape[0])\
    .rename(columns={0: 'Number of missed data  points', 'perc': '% of missed data points'})

,Number of missed data points,% of missed data points
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
5,0,0.0
...,...,...
57,0,0.0
58,0,0.0
59,0,0.0
60,0,0.0


# Adding features

In [62]:
def day_part(hour):
    part_of_day = 0
    if 0 <= hour <= 6:
        part_of_day = 1
    elif 6 < hour <= 12:
        part_of_day = 2
    elif 12 < hour <= 18:
        part_of_day = 3
    elif 18 < hour <= 23:
        part_of_day = 4
    return part_of_day

In [63]:
df = data_.copy(deep=True)

df["weekday"] = data_.index.weekday
df["dayofyear"] = data_.index.dayofyear
df["is_weekend"] = data_.index.weekday.isin([5, 6]).astype(np.int32)
df["month"] = data_.index.month
df["season"] = (data_.index.month % 12 + 3) // 3
df['Hour'] = df.index.hour
df['type_of_the_day'] = df.apply(lambda row : day_part(row['Hour']), axis = 1)

In [64]:
df_month = pd.get_dummies(df["month"], drop_first=True, prefix="month")
df_season = pd.get_dummies(df["season"], drop_first=True, prefix="season")
df_weekday = pd.get_dummies(df["weekday"], drop_first=True, prefix="weekday")
df_day_part = pd.get_dummies(df["type_of_the_day"], drop_first=True, prefix="day_part")
df = pd.concat([df, df_month, df_season, df_weekday,df_day_part], axis=1)
df.drop(columns=['month','season','weekday','type_of_the_day'], inplace=True)

In [65]:
rbf = RepeatingBasisFunction(n_periods=12,
                             column="dayofyear",
                             input_range=(1,365),
                             remainder="drop")

In [66]:
rbf.fit(df)
X_3 = pd.DataFrame(index=df.index,
                   data=rbf.transform(df))

In [67]:
X_3.columns = [str(c)+'_day_month' for c in range(len(X_3.columns))]

In [68]:
#df = pd.concat([df, X_3], axis=1)

In [69]:
def create_features(houshold_id,df=df) -> pd.DataFrame:
    df_new = df[[houshold_id, 'is_weekend',
                 #'month_2', 'month_3', 'month_4',
                 'month_5', 'month_6', 'month_7','month_8','month_9',
                 #'month_10','month_11','month_12',
                 #'season_2',
                 'season_3','season_4','weekday_1', 'weekday_2', 'weekday_3','weekday_4','weekday_5','weekday_6',
                 'day_part_2','day_part_3','day_part_4'
                 #'0_day_month','1_day_month','2_day_month','3_day_month','4_day_month','5_day_month','6_day_month','7_day_month','8_day_month','9_day_month','10_day_month','11_day_month'
                 ]]
    df_new['std'] = df_new[houshold_id].rolling(12).std().fillna(method="backfill")
    df_new['mean'] = df_new[houshold_id].rolling(12).mean().fillna(method="backfill")
    df_new['std'] = df_new[houshold_id].rolling(24).std().fillna(method="backfill")
    df_new['mean'] = df_new[houshold_id].rolling(24).mean().fillna(method="backfill")
    df_new['lag_1'] = df[houshold_id].shift(1).fillna(method="backfill")
    df_new['lag_2'] = df[houshold_id].shift(2).fillna(method="backfill")
    df_new['lag_3'] = df_new[houshold_id].shift(3).fillna(method="backfill")
    df_new['lag_4'] = df_new[houshold_id].shift(4).fillna(method="backfill")
    df_new['lag_5'] = df_new[houshold_id].shift(5).fillna(method="backfill")
    df_new['lag_6'] = df_new[houshold_id].shift(6).fillna(method="backfill")
    df_new['lag_7'] = df_new[houshold_id].shift(7).fillna(method="backfill")
    df_new['lag_8'] = df_new[houshold_id].shift(8).fillna(method="backfill")
    df_new['lag_9'] = df_new[houshold_id].shift(9).fillna(method="backfill")
    df_new['lag_10'] = df_new[houshold_id].shift(10).fillna(method="backfill")
    df_new['lag_11'] = df_new[houshold_id].shift(11).fillna(method="backfill")
    df_new['lag_12'] = df_new[houshold_id].shift(12).fillna(method="backfill")
    df_new['lag_13'] = df_new[houshold_id].shift(13).fillna(method="backfill")
    df_new['lag_14'] = df_new[houshold_id].shift(14).fillna(method="backfill")
    df_new['lag_15'] = df_new[houshold_id].shift(15).fillna(method="backfill")
    df_new['lag_16'] = df_new[houshold_id].shift(16).fillna(method="backfill")
    df_new['lag_17'] = df_new[houshold_id].shift(17).fillna(method="backfill")
    df_new['lag_18'] = df_new[houshold_id].shift(18).fillna(method="backfill")
    return df_new

In [70]:
def get_weeks(idx)-> List[List[pd._libs.tslibs.timestamps.Timestamp]]:
    idx = list(idx)
    count = 0
    hours_to_predict = []
    hours = []
    for i in range(len(idx)):
        if count > 167:
            hours_to_predict.append(hours)
            hours = []
            count = 0
        hours.append(idx[i])
        count += 1
    return hours_to_predict

In [71]:
def get_weeks_2(idx)-> List[List[pd._libs.tslibs.timestamps.Timestamp]]:
    idx = list(idx)
    weeks_to_predict = []
    week = []
    for i in range(1,len(idx)+1,1):
        #print(idx[i-1])
        if i%168 == 0:
            week.append(idx[i-1])
            weeks_to_predict.append(week)
            #print(week)
            week = []
        elif i%168 > 0:
            week.append(idx[i-1])
    return weeks_to_predict

In [72]:
hours_test_date = get_weeks_2(idx_test_date)

In [73]:
idx_test_date

DatetimeIndex(['2019-04-29 00:00:00', '2019-04-29 01:00:00',
               '2019-04-29 02:00:00', '2019-04-29 03:00:00',
               '2019-04-29 04:00:00', '2019-04-29 05:00:00',
               '2019-04-29 06:00:00', '2019-04-29 07:00:00',
               '2019-04-29 08:00:00', '2019-04-29 09:00:00',
               ...
               '2019-09-04 14:00:00', '2019-09-04 15:00:00',
               '2019-09-04 16:00:00', '2019-09-04 17:00:00',
               '2019-09-04 18:00:00', '2019-09-04 19:00:00',
               '2019-09-04 20:00:00', '2019-09-04 21:00:00',
               '2019-09-04 22:00:00', '2019-09-04 23:00:00'],
              dtype='datetime64[ns]', length=672, freq=None)

In [74]:
len(hours_test_date[0])

168

In [75]:
hours_test_date[0]

[Timestamp('2019-04-29 00:00:00'),
 Timestamp('2019-04-29 01:00:00'),
 Timestamp('2019-04-29 02:00:00'),
 Timestamp('2019-04-29 03:00:00'),
 Timestamp('2019-04-29 04:00:00'),
 Timestamp('2019-04-29 05:00:00'),
 Timestamp('2019-04-29 06:00:00'),
 Timestamp('2019-04-29 07:00:00'),
 Timestamp('2019-04-29 08:00:00'),
 Timestamp('2019-04-29 09:00:00'),
 Timestamp('2019-04-29 10:00:00'),
 Timestamp('2019-04-29 11:00:00'),
 Timestamp('2019-04-29 12:00:00'),
 Timestamp('2019-04-29 13:00:00'),
 Timestamp('2019-04-29 14:00:00'),
 Timestamp('2019-04-29 15:00:00'),
 Timestamp('2019-04-29 16:00:00'),
 Timestamp('2019-04-29 17:00:00'),
 Timestamp('2019-04-29 18:00:00'),
 Timestamp('2019-04-29 19:00:00'),
 Timestamp('2019-04-29 20:00:00'),
 Timestamp('2019-04-29 21:00:00'),
 Timestamp('2019-04-29 22:00:00'),
 Timestamp('2019-04-29 23:00:00'),
 Timestamp('2019-04-30 00:00:00'),
 Timestamp('2019-04-30 01:00:00'),
 Timestamp('2019-04-30 02:00:00'),
 Timestamp('2019-04-30 03:00:00'),
 Timestamp('2019-04-

In [76]:
def data_split_accumlated(result, weeks, n, houshold) -> Tuple[Any, Any]:
    print(weeks[n])
    result_splitted = result[result.index < weeks[n][0]]
    result_splitted_features = result_splitted.drop([houshold], axis=1)
    result_splitted_target = result_splitted[houshold]
    #print("target: ", result_splitted_target)
    result_splitted_to_predict = result[(result.index >= weeks[n][0]) & (result.index <= weeks[n][167])]
    result_splitted_to_predict_features = result_splitted_to_predict.drop([houshold], axis=1)
    result_splitted_to_predict_target = result_splitted_to_predict[houshold]
    result_splitted_to_predict_target = result_splitted_to_predict_target.to_frame()

    return result_splitted_features, result_splitted_target, result_splitted_to_predict_features, result_splitted_to_predict_target

In [77]:
all_predictions = []
 
for houshold in data_.columns:
    houshold_predictions = []
    print("***** Houshold " + str(houshold) + " dataset created ****** ")
    for week in range(len(hours_test_date)):
        #print("week "+  str(week) + " splitting started")
        result = create_features(houshold)
        features, target, features_predict, target_predict = data_split_accumlated(result, hours_test_date, week, houshold)
        model_linear = LinearRegression()
        model_linear.fit(features, target)
        print("trained on " + str(week))
        prediction = model_linear.predict(features_predict)
        houshold_predictions.append(prediction)
        week_timestamps = hours_test_date[week]
        target_predict['predict'] = prediction
        df.loc[week_timestamps[0]:week_timestamps[6],houshold] = target_predict['predict']
        print("=============================================")
        print(prediction)
        #print(df.loc[week_timestamps[0]:week_timestamps[6],houshold])
        print("=============================================")
    all_predictions.append(houshold_predictions)

***** Houshold 1 dataset created ****** 
[Timestamp('2019-04-29 00:00:00'), Timestamp('2019-04-29 01:00:00'), Timestamp('2019-04-29 02:00:00'), Timestamp('2019-04-29 03:00:00'), Timestamp('2019-04-29 04:00:00'), Timestamp('2019-04-29 05:00:00'), Timestamp('2019-04-29 06:00:00'), Timestamp('2019-04-29 07:00:00'), Timestamp('2019-04-29 08:00:00'), Timestamp('2019-04-29 09:00:00'), Timestamp('2019-04-29 10:00:00'), Timestamp('2019-04-29 11:00:00'), Timestamp('2019-04-29 12:00:00'), Timestamp('2019-04-29 13:00:00'), Timestamp('2019-04-29 14:00:00'), Timestamp('2019-04-29 15:00:00'), Timestamp('2019-04-29 16:00:00'), Timestamp('2019-04-29 17:00:00'), Timestamp('2019-04-29 18:00:00'), Timestamp('2019-04-29 19:00:00'), Timestamp('2019-04-29 20:00:00'), Timestamp('2019-04-29 21:00:00'), Timestamp('2019-04-29 22:00:00'), Timestamp('2019-04-29 23:00:00'), Timestamp('2019-04-30 00:00:00'), Timestamp('2019-04-30 01:00:00'), Timestamp('2019-04-30 02:00:00'), Timestamp('2019-04-30 03:00:00'), Timest

In [78]:
weeks_columns = [week for weeks in hours_test_date for week in weeks]
weeks_columns

[Timestamp('2019-04-29 00:00:00'),
 Timestamp('2019-04-29 01:00:00'),
 Timestamp('2019-04-29 02:00:00'),
 Timestamp('2019-04-29 03:00:00'),
 Timestamp('2019-04-29 04:00:00'),
 Timestamp('2019-04-29 05:00:00'),
 Timestamp('2019-04-29 06:00:00'),
 Timestamp('2019-04-29 07:00:00'),
 Timestamp('2019-04-29 08:00:00'),
 Timestamp('2019-04-29 09:00:00'),
 Timestamp('2019-04-29 10:00:00'),
 Timestamp('2019-04-29 11:00:00'),
 Timestamp('2019-04-29 12:00:00'),
 Timestamp('2019-04-29 13:00:00'),
 Timestamp('2019-04-29 14:00:00'),
 Timestamp('2019-04-29 15:00:00'),
 Timestamp('2019-04-29 16:00:00'),
 Timestamp('2019-04-29 17:00:00'),
 Timestamp('2019-04-29 18:00:00'),
 Timestamp('2019-04-29 19:00:00'),
 Timestamp('2019-04-29 20:00:00'),
 Timestamp('2019-04-29 21:00:00'),
 Timestamp('2019-04-29 22:00:00'),
 Timestamp('2019-04-29 23:00:00'),
 Timestamp('2019-04-30 00:00:00'),
 Timestamp('2019-04-30 01:00:00'),
 Timestamp('2019-04-30 02:00:00'),
 Timestamp('2019-04-30 03:00:00'),
 Timestamp('2019-04-

In [79]:
def flatten(hous):
    return [week for weeks in hous for week in weeks]

In [80]:
flatened_predictions = []
for hous in all_predictions:
    flatened_predictions.append(flatten(hous))

In [81]:
df_from_list = pd.DataFrame([i for i in flatened_predictions], columns= [weeks_columns],index = final_index)

In [82]:
df_from_list.reset_index(inplace = True)

In [83]:
df_from_list.to_csv("./sample_submission_hourly_max_eval_new_features.csv", index = False)

In [84]:
df_from_list

,pseudo_id,2019-04-29 00:00:00,2019-04-29 01:00:00,2019-04-29 02:00:00,2019-04-29 03:00:00,2019-04-29 04:00:00,2019-04-29 05:00:00,2019-04-29 06:00:00,2019-04-29 07:00:00,2019-04-29 08:00:00,...,2019-09-04 14:00:00,2019-09-04 15:00:00,2019-09-04 16:00:00,2019-09-04 17:00:00,2019-09-04 18:00:00,2019-09-04 19:00:00,2019-09-04 20:00:00,2019-09-04 21:00:00,2019-09-04 22:00:00,2019-09-04 23:00:00
0,0x16cb02173ebf3059efdc97fd1819f14a2,91.051097,62.632919,53.523385,63.371293,51.860389,36.377156,39.272849,47.025087,38.507665,...,4.577498e+01,4.577498e+01,4.577498e+01,4.577498e+01,4.577498e+01,4.579629e+01,4.579629e+01,4.579629e+01,4.579629e+01,4.579629e+01
1,0x1c9d08cd16fce04790ef900695861e786,6.292671,4.344006,3.819284,4.296919,3.518616,3.052310,2.920190,2.364203,1.680194,...,-2.226069e+12,-2.226069e+12,-2.226069e+12,-2.226069e+12,-2.226069e+12,-2.226069e+12,-2.226069e+12,-2.226069e+12,-2.226069e+12,-2.226069e+12
2,0x1612e4cbe3b1b85c3dbcaeaa504ee8424,17.272523,11.104120,8.001019,7.593936,6.183051,6.017259,7.068803,5.759943,4.424419,...,1.153491e+01,1.153491e+01,1.153491e+01,1.153491e+01,1.153491e+01,1.145986e+01,1.145986e+01,1.145986e+01,1.145986e+01,1.145986e+01
3,0x20158d36236a640cf0524dba149459169,107.009045,73.114554,54.776501,64.120198,60.819117,43.353297,39.873978,53.256463,40.905533,...,6.113579e+01,6.113579e+01,6.113579e+01,6.113579e+01,6.113579e+01,6.119773e+01,6.119773e+01,6.119773e+01,6.119773e+01,6.119773e+01
4,0xc305005dcb1ed6128d816954c5ab9e7e,60.901803,37.770439,31.951342,33.612381,20.540433,23.842822,27.422555,19.601114,19.728018,...,2.596984e+01,2.596984e+01,2.596984e+01,2.596984e+01,2.596984e+01,2.591592e+01,2.591592e+01,2.591592e+01,2.591592e+01,2.591592e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0x12342fbadc0ca9418f2d540bb3cb8364a,1.784231,1.545833,1.561398,1.545694,1.164337,1.081254,0.938577,0.712957,0.910837,...,4.518778e+10,4.518778e+10,4.518778e+10,4.518778e+10,4.518778e+10,4.518778e+10,4.518778e+10,4.518778e+10,4.518778e+10,4.518778e+10
57,0x16d1816bc5d185c47de080d4c6a64bc9a,4.696431,3.335775,3.484305,4.091321,3.642325,3.088896,3.458324,3.871644,3.651224,...,-9.233683e+09,-9.233683e+09,-9.233683e+09,-9.233683e+09,-9.233683e+09,-9.233683e+09,-9.233683e+09,-9.233683e+09,-9.233683e+09,-9.233683e+09
58,0x14f480f24c435af1b8574c1c6bab38a1c,8.411252,5.144771,4.094064,4.878116,4.567612,3.792630,4.038330,3.989670,3.303633,...,-1.412764e+12,-1.412764e+12,-1.412764e+12,-1.412764e+12,-1.412764e+12,-1.412764e+12,-1.412764e+12,-1.412764e+12,-1.412764e+12,-1.412764e+12
59,0x1c7fc724d0a4f89ed1de8a0a4b302db22,1.912191,1.463882,1.210984,0.961525,0.864876,0.806141,0.734316,0.762249,0.747163,...,-3.061474e+11,-3.061474e+11,-3.061474e+11,-3.061474e+11,-3.061474e+11,-3.061474e+11,-3.061474e+11,-3.061474e+11,-3.061474e+11,-3.061474e+11


In [85]:
import eli5
from eli5.sklearn import PermutationImportance

In [86]:
feature_names = list(features.columns)

In [87]:
features

,is_weekend,month_5,month_6,month_7,month_8,month_9,season_3,season_4,weekday_1,weekday_2,...,lag_9,lag_10,lag_11,lag_12,lag_13,lag_14,lag_15,lag_16,lag_17,lag_18
2019-04-01 00:00:00,0,0,0,0,0,0,0,0,0,0,...,7.212,7.212,7.212,7.212,7.212,7.212,7.212,7.212,7.212,7.212
2019-04-01 01:00:00,0,0,0,0,0,0,0,0,0,0,...,7.212,7.212,7.212,7.212,7.212,7.212,7.212,7.212,7.212,7.212
2019-04-01 02:00:00,0,0,0,0,0,0,0,0,0,0,...,7.212,7.212,7.212,7.212,7.212,7.212,7.212,7.212,7.212,7.212
2019-04-01 03:00:00,0,0,0,0,0,0,0,0,0,0,...,7.212,7.212,7.212,7.212,7.212,7.212,7.212,7.212,7.212,7.212
2019-04-01 04:00:00,0,0,0,0,0,0,0,0,0,0,...,7.212,7.212,7.212,7.212,7.212,7.212,7.212,7.212,7.212,7.212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-28 19:00:00,0,0,0,0,1,0,1,0,0,1,...,2.281,1.298,1.557,2.340,1.096,0.000,2.443,2.223,2.290,0.000
2019-08-28 20:00:00,0,0,0,0,1,0,1,0,0,1,...,2.835,2.281,1.298,1.557,2.340,1.096,0.000,2.443,2.223,2.290
2019-08-28 21:00:00,0,0,0,0,1,0,1,0,0,1,...,3.423,2.835,2.281,1.298,1.557,2.340,1.096,0.000,2.443,2.223
2019-08-28 22:00:00,0,0,0,0,1,0,1,0,0,1,...,2.359,3.423,2.835,2.281,1.298,1.557,2.340,1.096,0.000,2.443


In [88]:
eli5.explain_weights(model_linear, top=40, feature_names = feature_names)

Weight?,Feature
+7832812920644.801,season_3
+3915973354211.588,is_weekend
+52706390602.779,season_4
+1.660,mean
+0.255,lag_1
+0.236,std
+0.085,lag_2
+0.018,weekday_4
+0.018,lag_5
+0.005,month_5


In [89]:
perm = PermutationImportance(model_linear).fit(features, target)
eli5.show_weights(perm,top=50,feature_names = feature_names)

Weight,Feature
3808782000315943171915776.0000 ± 128071909864710135087104.0000,season_3
2591055737648970442211328.0000 ± 107263844620658257428480.0000,month_7
2528115112442862856306688.0000 ± 70499952182136873680896.0000,month_6
2373386075478007109124096.0000 ± 98746996130438392578048.0000,month_8
787457795346958936702976.0000 ± 28028808969361066819584.0000,is_weekend
479379135125181735895040.0000 ± 10828210392322344484864.0000,weekday_6
472824270014080207552512.0000 ± 21201946251820498157568.0000,weekday_5
4.2862 ± 0.1291,mean
0.1312 ± 0.0041,lag_1
0.0680 ± 0.0029,lag_18
